In [1]:
import os
import sys
import gc

# прописываем нужный нам путь к проекту
from pathlib import Path
PATH = Path('/home/vtrokhymenko/git/proj_news_viz/nlp/')
sys.path.append(str(PATH))

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import multiprocessing as mp
from multiprocessing import Pool
num_cores = mp.cpu_count() # number of cores on your machine; 20
num_partitions = num_cores * 2 # number of partitions to split dataframe

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

---
возьмем для примера датасет [lenta.ru](https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta)

для примера взят датасет, скаченный по линку с [gitHub](https://github.com/yutkin/Lenta.Ru-News-Dataset/tree/v1.1)

In [4]:
df = pd.read_csv(PATH / 'data/external/lenta-ru-news.csv')
print(df.shape)
df.sample(2)

(800975, 6)


,url,title,text,topic,tags,date
429638,https://lenta.ru/news/2012/07/30/train1/,При столкновении поезда с автобусом в Польше п...,В результате столкновения поезда с микроавтобу...,Бывший СССР,Все,2012/07/30
102244,https://lenta.ru/news/2004/09/29/letter/,Сто влиятельных западных политиков призвали НА...,Сто влиятельных политиков и интеллектуалов США...,Мир,Все,2004/09/29


In [5]:
# удалим пустые записи и дубликаты по признаку `text`
df.dropna(inplace=True)
df.text.drop_duplicates(inplace=True)

print(f'df.shape: {df.shape}')
df.isnull().sum()

df.shape: (712654, 6)


url      0
title    0
text     0
topic    0
tags     0
date     0
dtype: int64

In [6]:
# конкатенируем признаки `title` & `text` через символ '|'
df['full_text'] = df.title + ' | ' + df.text
df.sample(2)

,url,title,text,topic,tags,date,full_text
256163,https://lenta.ru/news/2008/11/21/chariot/,Болгарские археологи нашли фракийскую колесницу,При раскопках фракийского погребального курган...,Культура,Все,2008/11/21,Болгарские археологи нашли фракийскую колесниц...
18522,https://lenta.ru/news/2000/12/19/maple/,Владимир Путин посадил в Канаде сахарный клен,Президент России Владимир Путин в понедельник ...,Россия,Все,2000/12/19,Владимир Путин посадил в Канаде сахарный клен ...


---
препроцессим

In [7]:
from news_viz.data.preprocessing_tools import clean_text, lemmatize
# pip install pymorphy2

* почистим немного текст

In [8]:
def multiply_clean_text(data):
    data['clean_text'] = data['full_text'].apply(clean_text)
    return data

In [9]:
%%time

df = parallelize_dataframe(df, multiply_clean_text)
df = df[df.clean_text!=9999]

gc.collect()

CPU times: user 19.9 s, sys: 14.8 s, total: 34.6 s
Wall time: 34.9 s


0

* оставим только нужные нам столбцы

In [13]:
print(df.shape)
df.sample(1)

(711725, 8)


,url,title,text,topic,tags,date,full_text,clean_text
225503,https://lenta.ru/news/2008/04/16/zimbabwe1/,Инфляция в Зимбабве достигла 165 тысяч процент...,Инфляция в Зимбабве достигла очередного рекорд...,Экономика,Все,2008/04/16,Инфляция в Зимбабве достигла 165 тысяч процент...,инфляция зимбабве достигла тысяч процентов год...


In [14]:
df = df[['url','clean_text']]
print(df.shape)
df.sample(2)

(711725, 2)


,url,clean_text
219110,https://lenta.ru/news/2008/02/29/abel1/,суд решил депортировать владимира абеля из рос...
499410,https://lenta.ru/news/2014/03/04/nocooperation/,сша заморозили военное сотрудничество россией ...


* лемматизируем и удаляем стоп-слова (624 примера)

In [15]:
def multiply_lemmatization(data):
    data['lemma_text'] = data['clean_text'].apply(lemmatize)
    return data

In [16]:
%%time

df = parallelize_dataframe(df, multiply_lemmatization)
df = df[df.lemma_text!=9999]
gc.collect()

CPU times: user 14 s, sys: 10.5 s, total: 24.5 s
Wall time: 12min 28s


0

In [17]:
print(df.shape)
df.sample(2)

(711724, 3)


,url,clean_text,lemma_text
737116,https://lenta.ru/news/2018/11/30/syria_video/,авиаудар израиля по сирии попал на видео сирий...,авиаудар израиль сирия попасть видео сирийский...
348579,https://lenta.ru/news/2010/10/12/mccartney/,пол маккартни объявил конкурс каверов бывший у...,половина маккартень объявить конкурс кавер быв...


---

In [18]:
df[['url','lemma_text']].to_csv(f'{PATH}/data/interim/lenta_ru_news-cleaned-lemmatize.csv.gz', compression='gzip', index=False)